In [97]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
ANTHROPIC_API_KEY = <YOUR_API_KEY>
os.environ["LANGCHAIN_API_KEY"] = ANTHROPIC_API_KEY

In [98]:
agent_instructions = """You are a helpful assistant. Help the user answer any questions.

You have access to the following tools:

{tools}

In order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. \
You will then get back a response in the form <observation></observation>
For example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:

<tool>search</tool><tool_input>weather in SF</tool_input>
<observation>64 degrees</observation>

When you are done, respond with a final answer between <final_answer></final_answer>. For example:

<final_answer>The weather in SF is 64 degrees</final_answer>

Begin!

Question: {question}"""

In [99]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate, AIMessagePromptTemplate
from langchain.agents import tool, load_tools, Tool
from langchain.utilities import GoogleSerperAPIWrapper
from tempfile import TemporaryDirectory

working_directory = TemporaryDirectory()



In [100]:
model = ChatAnthropic(model="claude-2",max_tokens_to_sample=2000)

In [101]:
prompt_template = ChatPromptTemplate.from_template(agent_instructions) + AIMessagePromptTemplate.from_template("{intermediate_steps}")

In [102]:
chain = prompt_template | model.bind(stop=["</tool_input>", "</final_answer>"])

In [201]:
search = GoogleSerperAPIWrapper()
!pip install wget
import wget

import requests

from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)

tool_list = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to ask with search (especially for a research paper)",
    ),
    Tool(
        name="readFile",
        func=ReadFileTool().run,
        description="useful for when you need to read a file",
    ),
    Tool(
        name="downloadFile",
        func=wget.download,
        description="useful for when you need to download a file from a url",
    )
]


from langchain.agents.agent_toolkits import FileManagementToolkit

tools = FileManagementToolkit(
    root_dir=str(working_directory.name),
    selected_tools=["read_file", "write_file", "list_directory"],
).get_tools()



  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=f0785863aa63a520c3e2ddcc4bb48c86f19a55585f1026ee8d5c37a708f3b9ae
  Stored in directory: /Users/jay/Library/Caches/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [202]:
tools

[ReadFileTool(name='read_file', description='Read file from disk', args_schema=<class 'langchain.tools.file_management.read.ReadFileInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, root_dir='/var/folders/12/0zyls8316qd2mbd6n28kc9bm0000gn/T/tmpof74j_xr'),
 WriteFileTool(name='write_file', description='Write file to disk', args_schema=<class 'langchain.tools.file_management.write.WriteFileInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, root_dir='/var/folders/12/0zyls8316qd2mbd6n28kc9bm0000gn/T/tmpof74j_xr'),
 ListDirectoryTool(name='list_directory', description='List files and directories in a specified folder', args_schema=<class 'langchain.tools.file_management.list_dir.DirectoryListingInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=Fals

In [203]:
tool_list

[Tool(name='search', description='useful for when you need to ask with search (especially for a research paper)', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method GoogleSerperAPIWrapper.run of GoogleSerperAPIWrapper(k=10, gl='us', hl='en', type='search', tbs=None, serper_api_key='12791d683d28e8c5f696fa10ac9687c7fe73e315', aiosession=None, result_key_for_type={'news': 'news', 'places': 'places', 'images': 'images', 'search': 'organic'})>, coroutine=None),
 Tool(name='readFile', description='useful for when you need to read a file', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method BaseTool.run of ReadFileTool(name='read_file', description='Read file from disk', args_schema=<class 'langchain.tools.file_management.read.ReadFileInput'>, return_direct=False, verbose=F

In [204]:
# tools = load_tools(["google-serper"], llm=model)


In [205]:
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish


class AnthropicAgent(BaseSingleActionAgent):
    
    tools: List[Tool]
    chain: Any

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        log = ""
        for action, observation in intermediate_steps:
            log += f"<tool>{action.tool}</tool><tool_input>{action.tool_input}</tool_input><observation>{observation}</observation>"
        tools = ""
        for tool in self.tools:
            tools += f"{tool.name}: {tool.description}\n"
        response = self.chain.invoke({"intermediate_steps": log, "tools": tools, "question": kwargs["input"]})
        if "</tool>" in response.content:
            t, ti = response.content.split("</tool>")
            _t = t.split("<tool>")[1]
            _ti = ti.split("<tool_input>")[1]
            return AgentAction(tool=_t, tool_input=_ti, log=response.content)
        elif "<final_answer>" in response.content:
            t, ti = response.content.split("<final_answer>")
            return AgentFinish(return_values={"output": ti}, log=response.content)
        else:
            raise ValueError

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        raise ValueError

In [206]:
from langchain.document_loaders import UnstructuredURLLoader


agent = AnthropicAgent(tools=tool_list, chain=chain)

In [207]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

In [208]:
agent_executor.run("whats the arxiv url of the GPT-3 paper? Save the file after you find the url.")



> Entering new AgentExecutor chain...
 <tool>search</tool>
<tool_input>arxiv GPT-3 paperSpecifically, we train GPT-3, an autoregressive language model with 175 billion parameters, 10x more than any previous non-sparse language ... Cite as: arXiv:2005.14165. This paper provides an introductory survey to GPT-3. We cover some of the historical development behind this technology, some of the key ... In this paper, we test this hypothesis by training a 175 billion parameter autoregressive language model, which we call. GPT-3 ... To conduct a comprehensive analysis of the capabilities of GPT series models, we select six representative models, comprising two GPT-3 series ... Cite as: arXiv:2303.10420. In this paper we explore the capabilities of the most capable GPT-3 model, text-davinci-003, on an established statutory-reasoning dataset ... In this paper, we investigate the few-shot transfer learning abilities of GPT-3 in biomedical. NLP (BioNLP) tasks. This helps to understand whether GPT

'The arxiv url of the GPT-3 paper is https://arxiv.org/pdf/2005.14165.pdf. I found the url by searching for "arxiv GPT-3 paper", and then downloaded the pdf file.'

In [211]:
# BASE TOOLS TO ADD
# * check if papers are good
# * read pdf as text
# * code execution

# state variables
# * files in directory
# * path in folder
# * 

SyntaxError: invalid syntax (4052576605.py, line 2)